In [1]:
from keras import models
import pandas as pd 
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as pyplot
from sklearn.model_selection import train_test_split
from numpy.random import seed
import os
import time
import tensorflow as tf
import pandas as pd
import keras.backend as K
from csv import writer
from csv import reader

In [2]:
seeds = list(range(0,10))
multiply= [0.5,1,2]
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [3]:
data = pd.read_csv('.\Sonar_seed_result/Sonar.csv')

X = data.values[:,0:60].astype(float)
y = data.values[:,60]

encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

In [4]:
data

207

In [ ]:
for mltp in multiply:
    for actual_seed in seeds:
        seed(actual_seed)
        tf.random.set_seed(actual_seed)


        x_train,x_validate,y_train,y_validate = train_test_split(X,encoded_Y,test_size = 0.2,random_state = actual_seed)

        model = Sequential()
        model.add(Dense(100*mltp, input_dim=60, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))

        filename = ".\Sonar_seed_result/"+"Sonar_" + str(mltp) + "_" + str(actual_seed) + ".csv"
        history_logger= keras.callbacks.CSVLogger(filename, separator=",", append=False)
        time_callback = TimeHistory()
        model.compile(loss='binary_crossentropy', optimizer='adam')

        history=model.fit(x_train,y_train,batch_size=5, 
                          epochs=100,
                          validation_data=(x_validate,y_validate),
                          callbacks=[history_logger,time_callback]
                         )
        times = time_callback.times
        csv_input = pd.read_csv(filename)
        csv_input['time'] = times
        csv_input.to_csv(filename, index=False)
        K.clear_session()
        del model
        tf.compat.v1.reset_default_graph()